## Loading source files

In [1]:
library(MCPanel)
library(glue)
library(gtools)
library(rjson)

## Check which senario we are using

In [2]:
senario <- fromJSON(file = "src/senario_selection.json")$senario
T0 <- fromJSON(file = "src/senario_selection.json")$T0
print(glue('We are using senario ', senario, ' with T0 = {T0}'))

We are using senario D with T0 = 24


## Loading data and perform leave-one-out cross-validation in control units

In [3]:
# Path to control data.
path_control <- glue('src/Senario{senario}/control/')

# List the filenames for data.
filenames_control <- mixedsort(list.files(path = path_control))
sample_size <- length(filenames_control)

for(sample_idx in 1:sample_size){
  print(glue("{sample_idx}th implementation starts"))
  flush.console()
  control_sample <- glue(path_control, 'sim_data_control_{sample_idx}.csv')
  control_data <- read.csv(control_sample)
  
  # Rolling data: roll the first unit to the last row, and implement MC method to estimate.
  N <- nrow(control_data)
  T <- ncol(control_data)
  
  mc_fake_hat <- matrix(0, N, (T-T0))
  for(i in 1:N){
    control_roll <- rbind(control_data[(-1):(-i),], control_data[1:i,])
    rownames(control_roll) <- 1:N
    
    treat_mat <- matrix(1, N, T)
    treat_mat[N, (T0 + 1):T] <- 0
    fake_obs <- control_roll * treat_mat
    
    # Implement MC.
    est_model_MCPanel <- mcnnm_cv(as.matrix(control_roll), treat_mat, to_estimate_u = 1, to_estimate_v = 1)
    Mhat <- est_model_MCPanel$L + replicate(T,est_model_MCPanel$u) + t(replicate(N,est_model_MCPanel$v))
    
    mc_fake_hat[i,] <- Mhat[N, (T0+1):T]
  }
  
  write.table(mc_fake_hat, file=glue('meta_learner/ensemble_mc/mc_hat_temp_{sample_idx}.csv'),
              row.names=FALSE, col.names=TRUE, sep=',')
}

1th implementation starts
2th implementation starts
3th implementation starts
4th implementation starts
5th implementation starts
6th implementation starts
7th implementation starts
8th implementation starts
9th implementation starts
10th implementation starts
11th implementation starts
12th implementation starts
13th implementation starts
14th implementation starts
15th implementation starts
16th implementation starts
17th implementation starts
18th implementation starts
19th implementation starts
20th implementation starts
21th implementation starts
22th implementation starts
23th implementation starts
24th implementation starts
25th implementation starts
26th implementation starts
27th implementation starts
28th implementation starts
29th implementation starts
30th implementation starts
31th implementation starts
32th implementation starts
33th implementation starts
34th implementation starts
35th implementation starts
36th implementation starts
37th implementation starts
38th imple

Click here to go to [05 Stacking base learners](05%20Stacking%20base%20learners.ipynb)